# Convolutional Neural Network

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from util import placeholders,get_mnist,suppress_tf_warning,mlp,gpu_sess
%matplotlib inline  
%config InlineBackend.figure_format='retina'
suppress_tf_warning()
print ("TF version:[%s]."%(tf.__version__))

TF version:[1.15.0].


### Dataset

In [2]:
x_train,y_train,x_test,y_test = get_mnist()
n_train,n_test,x_dim,y_dim = x_train.shape[0],x_test.shape[0],\
    x_train.shape[1],y_train.shape[1]
print ("n_train:[%d], n_test:[%d], x_dim:[%d], y_dim:[%d]"%
       (n_train,n_test,x_dim,y_dim))

n_train:[60000], n_test:[10000], x_dim:[784], y_dim:[10]


### Model

In [3]:
ph_x = tf.placeholder(dtype=tf.float32,shape=[None,784],name='x')
ph_y = tf.placeholder(dtype=tf.float32,shape=[None,10],name='y')
ph_is_train = tf.placeholder(tf.bool,name='is_train') 

### Reshape

In [4]:
net = tf.reshape(ph_x,shape=[-1,28,28,1])
net

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

### Conv -> BN -> actv

In [5]:
net = tf.layers.conv2d(inputs=net,filters=32,kernel_size=3,padding='same',
                       activation=None)
net = tf.layers.max_pooling2d(inputs=net,pool_size=2,strides=2)
net = tf.layers.batch_normalization(net,training=ph_is_train)
net = tf.nn.relu(net)
net

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 32) dtype=float32>

In [6]:
net = tf.layers.conv2d(inputs=net,filters=32,kernel_size=3,padding='same',
                       activation=None)
net = tf.layers.max_pooling2d(inputs=net,pool_size=2,strides=2)
net = tf.layers.batch_normalization(net,training=ph_is_train)
net = tf.nn.relu(net)
net

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 32) dtype=float32>

### Flatten

In [7]:
net = tf.layers.flatten(net)
net

<tf.Tensor 'flatten/Reshape:0' shape=(?, 1568) dtype=float32>

### MLP

In [9]:
net = mlp(net,h_dims=[128,10],actv=tf.nn.relu,out_actv=None,
          USE_DROPOUT=True,ph_is_training=ph_is_train)
net

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 10) dtype=float32>

In [10]:
y_hat = net

### Graph

In [11]:
costs = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ph_y,logits=y_hat)
cost = tf.reduce_mean(costs) 
update_ops = tf.compat.v1.get_collection(tf.GraphKeys.UPDATE_OPS) # BN
optm = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
optm = tf.group([optm, update_ops])
corr = tf.equal(tf.argmax(y_hat,1),tf.argmax(ph_y,1)) # [N]
accr = tf.reduce_mean(tf.cast(corr, "float")) # [1]

In [12]:
sess = gpu_sess()

In [13]:
def get_accr(sess,x,y):
    n,batch_size = x.shape[0],256
    accr_val_sum = 0.0
    for it in range(np.ceil(n/batch_size).astype(np.int)):
        x_batch = x[it*batch_size:(it+1)*batch_size,:]
        y_batch = y[it*batch_size:(it+1)*batch_size,:]
        feeds = {ph_x:x_batch,ph_y:y_batch,ph_is_train:False}
        accr_val = sess.run(accr,feed_dict=feeds)
        accr_val_sum += accr_val*x_batch.shape[0]
    accr_val_avg = accr_val_sum/n
    return accr_val_avg

### Loop

In [14]:
sess.run(tf.global_variables_initializer()) # Initialize variables
max_epoch,batch_size,print_every = 20,128,1
max_iter = np.ceil(n_train/batch_size).astype(np.int) # number of iterations
for epoch in range(max_epoch):
    p_idx = np.random.permutation(n_train)
    for it in range(max_iter):
        b_idx = p_idx[batch_size*(it):batch_size*(it+1)]
        x_batch,y_batch = x_train[b_idx,:],y_train[b_idx,:]
        feeds = {ph_x:x_batch,ph_y:y_batch,ph_is_train:True}
        sess.run(optm,feed_dict=feeds)
    
    if ((epoch%print_every)==0) or (epoch==(max_epoch-1)):
        train_accr_val = get_accr(sess,x_train,y_train)
        test_accr_val = get_accr(sess,x_test,y_test)
        print ("epoch:[%d/%d] train_accuracy:[%.3f] test_accuracy:[%.3f]"%
               (epoch,max_epoch,train_accr_val,test_accr_val))
print ("Done.")

epoch:[0/20] train_accuracy:[0.975] test_accuracy:[0.974]
epoch:[1/20] train_accuracy:[0.988] test_accuracy:[0.987]
epoch:[2/20] train_accuracy:[0.992] test_accuracy:[0.989]
epoch:[3/20] train_accuracy:[0.991] test_accuracy:[0.988]
epoch:[4/20] train_accuracy:[0.994] test_accuracy:[0.990]
epoch:[5/20] train_accuracy:[0.995] test_accuracy:[0.992]
epoch:[6/20] train_accuracy:[0.995] test_accuracy:[0.992]
epoch:[7/20] train_accuracy:[0.996] test_accuracy:[0.990]
epoch:[8/20] train_accuracy:[0.993] test_accuracy:[0.988]
epoch:[9/20] train_accuracy:[0.997] test_accuracy:[0.992]
epoch:[10/20] train_accuracy:[0.994] test_accuracy:[0.988]
epoch:[11/20] train_accuracy:[0.997] test_accuracy:[0.992]
epoch:[12/20] train_accuracy:[0.998] test_accuracy:[0.992]
epoch:[13/20] train_accuracy:[0.998] test_accuracy:[0.993]
epoch:[14/20] train_accuracy:[0.993] test_accuracy:[0.990]
epoch:[15/20] train_accuracy:[0.998] test_accuracy:[0.992]
epoch:[16/20] train_accuracy:[0.998] test_accuracy:[0.993]
epoch:[